In [1]:
import torch
from torchtext import data, datasets
import random

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
from torchtext import data
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import string
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [3]:
%config IPCompleter.greedy = True
SEED = 0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

## 1. RNN for Language Modeling (40pt)

#### 1.1. Import the torchtext IMDB dataset

In [4]:
def generate_trigrams(x):
    res = []
    n_grams = list(zip(*[x[i:] for i in range(3)]))
    for n_gram in n_grams:
        res.append(' '.join(n_gram))
    return res

In [5]:
import spacy


In [6]:
nlp = spacy.load("en_core_web_sm")

In [7]:
# Download pre-split train and test data

TEXT = data.Field(tokenize= nlp,preprocessing=generate_trigrams) #default parse by spaces
LABEL = data.LabelField(dtype = torch.float)
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
print(vars(train_data.examples[0]))
train_data, valid_data = train_data.split(random_state=random.seed(SEED),split_ratio=0.9)
TEXT.build_vocab(train_data)
dictionary = dict(TEXT.vocab.freqs)

TypeError: sequence item 0: expected str instance, spacy.tokens.token.Token found

In [ ]:
print('Train Samples: {}, of type: {}'.format(len(train_data), type(train_data.examples[0])))
print(vars(train_data.examples[0]))
print()
print('Test Samples: {}, of type: {}'.format(len(test_data), type(test_data.examples[0])))
print(vars(test_data.examples[0]))

In [ ]:
# Split train data into train and validation data, default split is 70/30
train_data, valid_data = train_data.split(random_state = random.seed(SEED))
print('Train Samples: {}, of type: {}'.format(len(train_data), type(train_data.examples[0])))
print('Valid Samples: {}, of type: {}'.format(len(valid_data), type(valid_data.examples[0])))

#### 1.2. Build a Markov (n-gram) language model

In [ ]:
# Build a vocabulary of most common words in the train set
VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size = VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [ ]:
print(vars(train_data.examples[0]))
print('TEXT Vocab Size:', len(TEXT.vocab))
print('LABEL Vocab Size:',len(LABEL.vocab))

In [ ]:

markov = pd.DataFrame.from_dict(dictionary,orient="index")
markov = markov.reset_index()
markov.columns = ['trigrams','cnt']
markov['bigram'] =markov.trigrams.apply(lambda x: " ".join(x.split(' ')[:2]))
markov['target'] =markov.trigrams.apply(lambda x: x.split(' ')[2])
inp_cnt = pd.DataFrame(markov.groupby("bigram",as_index=False)["cnt"].sum())

#### 1.3. Change the output appropriately in ‘Simple Sentiment Analysis.ipynb’ to build an LSTM based language model. Plot the training performance as a function of epochs/iterations

#### 2. For each model, describe the key design choices made. Briefly mention how each choice influences training time and generative quality.

#### 3. For each model, starting with the phrase ”My favorite movie ”, sample the next few words and create a 20 word generated review. Repeat this 5 times (you should ideally get different outputs each time) and report the outputs.